In [1]:
import numpy as np
from model import *
import datetime
from utils import *
from preprocessing import *
import random
import glob
from os.path import join, getctime, basename
from load_data import load_data
from save_model import replace_model_if_better
from keras.models import load_model
from shutil import rmtree

## Loading Preprocessed Data

# Create result folders 
save_path = join(get_save_path(), 'VanillaRNN')
new_path = join(save_path, 'best_val')
time = str(datetime.datetime.now()).replace(' ', '_')
workpath = join(save_path, time)
ensure_dir(new_path)
ensure_dir(workpath)
ensure_dir(save_path)

# Load preprocessed data
aug_data = load_data_pickle(get_save_path())
total_X_test = aug_data['total_X_test']
total_y_test = aug_data['total_y_test']
total_X_train = aug_data['total_X_train']
total_y_train = aug_data['total_y_train']
total_X_val = aug_data['total_X_val']
total_y_val = aug_data['total_y_val']
total_X_train = np.transpose(total_X_train, (0, 2, 1))
total_X_val = np.transpose(total_X_val, (0, 2, 1))
total_X_test = np.transpose(total_X_test, (0, 2, 1))
print(total_X_train.shape)
print(total_y_train.shape)
print(total_X_test.shape)
print(total_y_test.shape)
print(total_X_val.shape)
print(total_y_val.shape)

Using TensorFlow backend.


Loading data pickle...
Data pickle loaded.
(11844, 100, 22)
(11844, 4)
(3101, 100, 22)
(3101, 4)
(2961, 100, 22)
(2961, 4)


In [2]:
config = {
    # Network
    'num_inputs': total_X_train.shape[0],
    'input_shape': (total_X_train.shape[1],total_X_train.shape[2],1),
    'epochs': 50,
    'dropout': 0.5,
    'batch_size': 512,
    'l2': 0.05,
    'LSTM': False,
    'lr': 0.001
}

In [3]:
VanillaRNN1 = VanillaRNN()
VanillaRNN1.build_model(config)
VanillaRNN1.train(total_X_train, total_y_train, total_X_val, total_y_val, config, workpath)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 100, 22)           2970      
_________________________________________________________________
flatten_1 (Flatten)          (None, 2200)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 8804      
Total params: 11,774
Trainable params: 11,774
Non-trainable params: 0
_________________________________________________________________
Model compiled.
Train on 11844 samples, validate on 2961 samples
Epoch 1/50
11844/11844 [==============================] - 1s 117us/step - loss: 1.3631 - accuracy: 0.3311 - val_loss: 1.3303 - val_accuracy: 0.3722
Epoch 2/50
11844/11844 [===========

Epoch 50/50
11844/11844 [==============================] - 1s 91us/step - loss: 0.8933 - accuracy: 0.8565 - val_loss: 1.1143 - val_accuracy: 0.6174


In [4]:
# Majority Voting
preds = [VanillaRNN1.predict(total_X_test[i::443], 0) for i in range(443)]
majority_pred = []
for pred in preds:
    arg_maxes = [np.argmax(p) for p in pred]
    votes = np.bincount(arg_maxes)
    out = np.ndarray(shape=(4,), buffer=np.zeros(4), dtype=int)
    out[votes.argmax()] = 1
    majority_pred.append(out)
result = [(a == b).all() for a, b in zip(majority_pred, total_y_test[0:443])]
raw = VanillaRNN1.evaluate(total_X_test, total_y_test)
print("Raw Acc result: {}".format(raw[1]))
print("Majority Vote result: {}".format(np.mean(result)))
filepath = join(save_path, 'VanillaGRU.pickle')
replace_model_if_better(filepath, np.mean(result), VanillaRNN, config)

3101/3101 [==============================] - 0s 118us/step
Raw Acc result: 0.6336665749549866
Majority Vote result: 0.6501128668171557
No existing model in specified path. Saving the new model


True

In [5]:
# Load the checkpointed model with highest val acc and perform majority voting
model_path = join(workpath, '*.hdf5')
list_of_files = glob.glob(model_path)
latest_file = max(list_of_files, key=getctime)
model_val = load_model(latest_file)
preds = [model_val.predict(total_X_test[i::443], verbose=0) 
         for i in range(443)]
majority_pred = []
for pred in preds:
    arg_maxes = [np.argmax(p) for p in pred]
    votes = np.bincount(arg_maxes)
    out = np.ndarray(shape=(4,), buffer=np.zeros(4), dtype=int)
    out[votes.argmax()] = 1
    majority_pred.append(out)
result = [(a == b).all() for a, b in zip(majority_pred, total_y_test[0:443])]
raw = model_val.evaluate(total_X_test, total_y_test)
print("Raw Acc result: {}".format(raw[1]))
print("Majority Vote result: {}".format(np.mean(result)))
filepath = join(save_path, 'best_val', 'VanillaRNN_val.pickle')
replaced = replace_model_if_better(filepath, np.mean(result), model_val, config)
rmtree(workpath)

3101/3101 [==============================] - 0s 136us/step
Raw Acc result: 0.6336665749549866
Majority Vote result: 0.6501128668171557
No existing model in specified path. Saving the new model


In [6]:
config = {
    # Network
    'num_inputs': total_X_train.shape[0],
    'input_shape': (total_X_train.shape[1],total_X_train.shape[2],1),
    'epochs': 50,
    'dropout': 0.5,
    'batch_size': 512,
    'l2': 0.05,
    'LSTM': True,
    'lr': 0.001
}

In [7]:
VanillaRNN2 = VanillaRNN()
VanillaRNN2.build_model(config)
VanillaRNN2.train(total_X_train, total_y_train, total_X_val, total_y_val, config, workpath)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 22)           3960      
_________________________________________________________________
flatten_2 (Flatten)          (None, 2200)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2200)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 8804      
Total params: 12,764
Trainable params: 12,764
Non-trainable params: 0
_________________________________________________________________
Model compiled.
Train on 11844 samples, validate on 2961 samples
Epoch 1/50
11844/11844 [==============================] - 1s 118us/step - loss: 1.3600 - accuracy: 0.3266 - val_loss: 1.3418 - val_accuracy: 0.3600
Epoch 2/50
11844/11844 [===========

Epoch 50/50
11844/11844 [==============================] - 1s 102us/step - loss: 0.8879 - accuracy: 0.8652 - val_loss: 1.1460 - val_accuracy: 0.5758


In [8]:
# Majority Voting
preds = [VanillaRNN2.predict(total_X_test[i::443], 0) for i in range(443)]
majority_pred = []
for pred in preds:
    arg_maxes = [np.argmax(p) for p in pred]
    votes = np.bincount(arg_maxes)
    out = np.ndarray(shape=(4,), buffer=np.zeros(4), dtype=int)
    out[votes.argmax()] = 1
    majority_pred.append(out)
result = [(a == b).all() for a, b in zip(majority_pred, total_y_test[0:443])]
raw = VanillaRNN2.evaluate(total_X_test, total_y_test)
print("Raw Acc result: {}".format(raw[1]))
print("Majority Vote result: {}".format(np.mean(result)))
filepath = join(save_path, 'VanillaRNN.pickle')
replace_model_if_better(filepath, np.mean(result), VanillaRNN2, config)

3101/3101 [==============================] - 0s 117us/step
Raw Acc result: 0.5707836151123047
Majority Vote result: 0.5846501128668171
No existing model in specified path. Saving the new model


True

In [9]:
# Load the checkpointed model with highest val acc and perform majority voting
model_path = join(workpath, '*.hdf5')
list_of_files = glob.glob(model_path)
latest_file = max(list_of_files, key=getctime)
model_val = load_model(latest_file)
preds = [model_val.predict(total_X_test[i::443], verbose=0) 
         for i in range(443)]
majority_pred = []
for pred in preds:
    arg_maxes = [np.argmax(p) for p in pred]
    votes = np.bincount(arg_maxes)
    out = np.ndarray(shape=(4,), buffer=np.zeros(4), dtype=int)
    out[votes.argmax()] = 1
    majority_pred.append(out)
result = [(a == b).all() for a, b in zip(majority_pred, total_y_test[0:443])]
raw = model_val.evaluate(total_X_test, total_y_test)
print("Raw Acc result: {}".format(raw[1]))
print("Majority Vote result: {}".format(np.mean(result)))
filepath = join(save_path, 'best_val', 'ConvMixGRU_val.pickle')
replaced = replace_model_if_better(filepath, np.mean(result), model_val, config)
rmtree(workpath)

3101/3101 [==============================] - 0s 132us/step
Raw Acc result: 0.5775556564331055
Majority Vote result: 0.5959367945823928
No existing model in specified path. Saving the new model
